In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:

import time
import numpy as np
import cv2
from matplotlib import pyplot as plt
import os
import csv

from os import listdir
from os.path import isfile, join

In [4]:
train_data_list = []
with open('drive/My Drive/korat_sauce/img_csv.csv') as f_img:
    img_bins = csv.reader(f_img)
    for im in img_bins:
        train_data_list.append(im)
    f_img.close()

# r_bins =  np.array(train_data_list[0][1:33]).astype(np.float)
# g_bins = np.array(train_data_list[0][33:65]).astype(np.float)
# b_bins = np.array(train_data_list[0][65:97]).astype(np.float)




In [5]:
r_bins = []
g_bins = []
b_bins = []
target = []
data_for_train = []
for i in range(len(train_data_list)):
    img_data = np.array(train_data_list[i]).astype(np.float)
    min_x =  min(img_data) 
    max_x = max(img_data)
    for j in range(1,len(img_data)):
        img_data[j] = (img_data[j]-min_x)/(max_x-min_x) #normalization
    data_for_train.append(img_data)
    r_bins.append(img_data[1:33])
    g_bins.append(img_data[33:65])
    b_bins.append(img_data[65:97])
    target.append(img_data[0])


bins_order = np.arange(32).astype(np.float)
for i in range(len(bins_order)):
    bins_order[i]  = (bins_order[i]-1)/31

In [6]:
def perceptron(input_data,weight,bias):
    #print(np.dot(weight,np.transpose(input_data))+bias)
    return np.dot(input_data,np.transpose(weight))+bias

In [7]:
def transfer_function(result_of_summation): 
    #print("result_of_summation = ",result_of_summation)
    for i in range(len(result_of_summation)):
        if(result_of_summation[i]>=2):
            result_of_summation[i] = 1
        else:
            result_of_summation[i] = 0
    return result_of_summation

In [56]:
def train_weight(input_i,weight,weight2,bias,target,result2,result1):
    target_array = 0
    if (target == 1):
          target_array = np.array([1,0,0,0])
    elif (target == 2):
          target_array = np.array([0,1,0,0])
    elif (target == 3):
          target_array = np.array([0,0,1,0])
    elif (target == 4):
          target_array = np.array([0,0,0,1])
    while not(np.array_equal(target_array,result2)):
        error = target_array-result2
        print(error)
        
        #print(sum(sum(weight2)))
        spread_error = np.dot(error,weight2)/sum(sum(weight2))*0.1
        print(spread_error)
        break
    #     weight = weight+np.dot(error,input_data)*0.1
    #     bias = bias+error*0.1
    #     result = transfer_function_for_separate_bins_line1(np.dot(weight,np.transpose(input_data))+bias)
    #     print("in train mode : error = ",error,"weight = ",weight," bias = ",bias," result = ",result)
    # return weight,bias,result

In [9]:
weight_r_1 = np.random.rand(16,32)
weight_g_1 = np.random.rand(16,32)
weight_b_1 = np.random.rand(16,32)

weight_r_2 = np.random.rand(4,16)
weight_g_2 = np.random.rand(4,16)
weight_b_2 = np.random.rand(4,16)

weight_r_3 = np.random.rand(4,4)
weight_g_3 = np.random.rand(4,4)
weight_b_3 = np.random.rand(4,4)

In [10]:
bias = np.ones(7)

In [44]:
#test for r_bin
def train_until_weight_suit_for_classify(input_data,weight,weight2,bias,target):
    round_for_train = 0
    check_true = 0

    while (check_true <32 and round_for_train<200):
        round_for_train+=1
        check_true = 0

        #นำ แต่ละ ช่วงbins กับความสูงของbins (เป็น x y) เข้าสู่ โหนดเส้นสูง
        #นำ แต่ละ ช่วงbins กับความสูงของbins (เป็น x y) เข้าสู่ โหนดเส้นกลาง
        #นำ แต่ละ ช่วงbins กับความสูงของbins (เป็น x y) เข้าสู่ โหนดเส้นต่ำ
        #นำ ค่า ที่ออกมาจากทั้ง 3 เส้น เข้า 4 โหนดเพื่อแยกชนิด(4 โหนดนี้เป็นตัวเฉลย)
        #เช็ค ผลลัพธ์ จากนั้นทำการ ฝึกกลับ(เมื่อผลลัพธ์ผิด) โดยเริ่มฝึกที่ส่วนของ ฟังก์ชัน 4 โหนด
        #ปรับ weight ของทาง จากเส้น สูง กลาง ต่ำ ต่อ
        for i in range(len(input_data)):
            result_red_chanal1 = transfer_function(perceptron(input_data[i],weight,bias[0]) )
            print(result_red_chanal1)
            result_red_chanal2 = transfer_function(perceptron(result_red_chanal1,weight2,bias[3]) )
            print(result_red_chanal2)
            train_weight(input_data[i],weight,weight2,bias,target[i],result_red_chanal2,result_red_chanal1)
            break
        break



    print("Now,Weight are OK! and use round for train = ",round_for_train)
    #return weight,bias